In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.ToTensor() #make a initial transform.

In [ ]:
#allocate training data and testing data.
train_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = transform

)

100%|██████████| 26.4M/26.4M [00:02<00:00, 9.19MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 162kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.09MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.63MB/s]


In [ ]:
#create a dataloader(s).

train_loader = DataLoader(
    train_data,
    batch_size=64,
    shuffle = True,
    num_workers=2, pin_memory=True
)

test_loader = DataLoader(
    test_data,
    batch_size = 64,
    shuffle=False,
    num_workers=2, pin_memory=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64,10)
)

In [ ]:
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
lossfunc = nn.CrossEntropyLoss()

In [ ]:
lossfunc

CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.cuda.memory_allocated())

True
0
Tesla T4
438272


In [ ]:
epochs = 100
running_loss = 0

for epoch in range(epochs):
  model.train()


  for images, labels in train_loader: #its the training batch we made earlier.

    images = images.to(device)
    labels = labels.to(device)

    #loss
    outputs = model(images)
    losses = lossfunc(outputs, labels)

    #Backpropagation
    optimizer.zero_grad() #clears gradient
    losses.backward() #backward propagation -- trying again.
    optimizer.step() #making new gradients.

    running_loss += losses.item()

  print(f"Epoch {epoch}/{epochs}, RunningLoss: {running_loss:.4f}, CurrentLoss({losses.item():.4f}), memoryallocated{torch.cuda.memory_allocated()}") #epoch out of epochs, loss running loss

print("Average loss per batch: {}".format(running_loss/epochs))


Epoch 0/100, RunningLoss: 530.7615, CurrentLoss(0.3393), memoryallocated19099648
Epoch 1/100, RunningLoss: 898.7869, CurrentLoss(0.2010), memoryallocated19099648
Epoch 2/100, RunningLoss: 1226.8123, CurrentLoss(0.2836), memoryallocated19099648
Epoch 3/100, RunningLoss: 1533.2661, CurrentLoss(0.3472), memoryallocated19099648
Epoch 4/100, RunningLoss: 1819.7913, CurrentLoss(0.4858), memoryallocated19099648
Epoch 5/100, RunningLoss: 2091.2678, CurrentLoss(0.3537), memoryallocated19099648
Epoch 6/100, RunningLoss: 2352.5838, CurrentLoss(0.1526), memoryallocated18895360
Epoch 7/100, RunningLoss: 2603.0363, CurrentLoss(0.4485), memoryallocated19099648
Epoch 8/100, RunningLoss: 2841.5908, CurrentLoss(0.3208), memoryallocated19099648
Epoch 9/100, RunningLoss: 3072.1734, CurrentLoss(0.1176), memoryallocated19099648
Epoch 10/100, RunningLoss: 3299.4228, CurrentLoss(0.1432), memoryallocated19099648
Epoch 11/100, RunningLoss: 3516.5069, CurrentLoss(0.2133), memoryallocated19099648
Epoch 12/100, Ru

In [ ]:
model.eval() #set the model to evaluation mode. Faster, more memory-efficient
correct = 0
total = 0

with torch.no_grad(): #turned off gradient tracking.
  for images, labels in test_loader: #used test_loader.
    images = images.to(device)
    labels = labels.to(device) #common setup.

    outputs = model(images) #forward pass, or prediction
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print("Test accuracy: {}".format(100 * correct / total))

Test accuracy: 88.35


In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
torch.save(model.state_dict(), '/content/drive/MyDrive/artificialintelligence/pytorchtest/model.pth')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
